## MSA 8040 Project III - Fall 2020
-- Aishat Olayiwola, Anitha Karunakaran, Patrick Silva, Weiting Yu

### Demo Code: Web Scraping and Data Extraction

In [11]:
#Importing the libraries
import csv
import time 
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys

import requests, re 
from bs4 import BeautifulSoup
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

from collections import Counter

import random
import sys

### Live demo of web scraping using a random URL from the following:

a.	https://seekingalpha.com/article/4393745-newmont-corporation-nem-ceo-tom-palmer-hosts-2021-investor-update-call-transcript

b.	https://seekingalpha.com/article/4393736-mongodb-inc-mdb-ceo-dev-ittycheria-on-q3-2021-results-earnings-call-transcript

c.	https://seekingalpha.com/article/4393631-barnes-noble-education-inc-bned-ceo-mike-huseby-on-q2-2021-results-earnings-call-transcript

d.	https://seekingalpha.com/article/4393377-lightinthebox-holding-co-ltd-s-litb-ceo-jian-on-q3-2020-results-earnings-call-transcript

e.	https://seekingalpha.com/article/4392903-idt-corporations-idt-ceo-shmuel-jonas-on-q1-2021-results-earnings-call-transcript

f.	https://seekingalpha.com/article/4392859-union-pacific-corporation-unp-ceo-lance-fritz-presents-credit-suisse-8th-annual-virtual

g.	https://seekingalpha.com/article/4392655-sarepta-therapeutics-inc-srpt-presents-evercore-isi-healthconx-conference-transcript

h.	https://seekingalpha.com/article/4392571-regeneron-pharmaceuticals-incs-regn-management-presents-piper-sandler-32nd-annual-virtual

i.	https://seekingalpha.com/article/4392384-twitter-inc-twtr-presents-bofa-2020-leveraged-finance-virtual-conference-transcript

j.	https://seekingalpha.com/article/4391728-laix-inc-s-laix-ceo-yi-wang-on-q3-2020-results-earnings-call-transcript

In [12]:
class MSA8040_SeekingAlphaBot:

    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.driver = webdriver.Firefox()

    def closeBrowser(self):
        self.browser.close()

    def login(self):
        #Visits the Website
        driver = self.driver
        driver.get("https://seekingalpha.com")
        time.sleep(2)
        
        #Click on Sign In Button and Proceeds with the Login
        login_button = driver.find_element_by_xpath("//button[@data-test-id='header-button-sign-in']")
        login_button.click()
        time.sleep(2)
        
        ##Login Parameters
        user_name_elem = driver.find_element_by_xpath("//input[@name='email']")
        user_name_elem.clear()
        user_name_elem.send_keys(self.username)
        passworword_elem = driver.find_element_by_xpath("//input[@name='password']")
        passworword_elem.clear()
        passworword_elem.send_keys(self.password)
        passworword_elem.send_keys(Keys.RETURN)
        time.sleep(2) 
        
        transcript_title = []
        transcript_content = []
          
        #------------------------------------ Insert your URL here ---------------------------------------------#
        base_url = "https://seekingalpha.com/article/4391728-laix-inc-s-laix-ceo-yi-wang-on-q3-2020-results-earnings-call-transcript"
            
        time.sleep(2) 
        driver.get(base_url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        all1=soup.find_all("div",{"class":"sa-art-hd"}) #title
        all2=soup.find_all("div",{"class":"sa-art article-width"}) #Content
            
        #------------------------------------ TITLE --------------------------------------------------------#
        for item in all1:

            # enter dictionary for store the data
            d1 = {}
            try:
                d1["Title"] = str(item.find("h1")).split(">")[1].split("<")[0]
                time.sleep(2)

            except Exception:
                continue

            transcript_title.append(d1)

            #------------------------------------ CONTENT --------------------------------------------------------#
        for item in all2:

            # enter dictionary for store the data
            d2 = {}
            try:
                d2["Content"] = item.text
                time.sleep(2)

            except Exception:
                continue

            transcript_content.append(d2)

         #-----------------------------------------------------------------------------------------------------#   
        df = pd.concat([pd.DataFrame(transcript_title), pd.DataFrame(transcript_content)], axis=1)
            
        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter('df_demo.xlsx', engine='xlsxwriter')
            
        # Convert the dataframe to an XlsxWriter Excel object.
        df.to_excel(writer, sheet_name='Sheet1')
            
        # Close the Pandas Excel writer and output the Excel file.
        writer.save()
        
Project3 = MSA8040_SeekingAlphaBot("patricks.s@hotmail.com","msa80402020")
Project3.login()       

### Data Extraction

In [9]:
df = pd.read_excel('df_demo.xlsx')

df

0    LAIX Inc.'s (LAIX) CEO Yi Wang on Q3 2020 Resu...
Name: Title, dtype: object
0    LAIX Inc. (NYSE:LAIX) Q3 2020 Earnings Confere...
Name: Content, dtype: object


,Unnamed: 0,Title,Content
0,0,LAIX Inc.'s (LAIX) CEO Yi Wang on Q3 2020 Resu...,LAIX Inc. (NYSE:LAIX) Q3 2020 Earnings Confere...


In [4]:
df = pd.read_excel('df_demo.xlsx')
prow = pd.DataFrame(columns = ['Ticker','Company','Title','Date','Time','Speech','Participant name'])

def run(context):
    df_list=context.split("\n")
    df_list =list(filter(lambda a: a != '', df_list ))
    def listToString(s):  

        # initialize an empty string 
        str1 = " " 
        # return string   
        return (str1.join(s)) 
    def find_company(df):
        Company=df_list[0].split('(')[0]
        Ticker=df_list[0].split('(')[1].split(')')[0].split(':')[1]
        a=df_list[0].split(')')[1].split(' ')[:-7]
        Conference=listToString(a)
        Month=df_list[0].split(')')[1].split(' ')[-7]
        Date=df_list[0].split(')')[1].split(' ')[-6]
        Year=df_list[0].split(')')[1].split(' ')[-5]
        Time=df_list[0].split(')')[1].split(' ')[-3]
        AM_PM=df_list[0].split(')')[1].split(' ')[-2]
        Time_Zone=df_list[0].split(')')[1].split(' ')[-1]
        df_Company= pd.DataFrame()
        df_Company['Ticker']=[Ticker]
        df_Company['Company']=[Company]
        df_Company['Conference']=[Conference]
        df_Company['Month']=[Month]
        df_Company['Date']=[Date]
        df_Company['Year']=[Year]
        df_Company['Time']=[Time]
        df_Company['AM_PM']=[AM_PM]
        df_Company['Time_Zone']=[Time_Zone]
        return df_Company
    find_company=find_company(df)
    def find_Company_Participants(df):
        i=0
        Company_Participants_list=[]
        Conference_Call_Participants_list=[]
        while i>=0:
            i=i+1
            if df_list[i]=='Conference Call Participants':
                break
            Company_Participants_list.append(i)   
        while i==i:
            i=i+1
            if len(df_list[i])>100:
                break
            Conference_Call_Participants_list.append(i)
        Company_Participants_list=Company_Participants_list[1:]
        Conference_Call_Participants_list=Conference_Call_Participants_list[:-1]
        Company_Participants_name=[]
        Company_Participants_type=[]
        for i in Company_Participants_list:
            name_title=df_list[i]
            Company_Participants_name.append((name_title.split('-')[0]))
            Company_Participants_type.append(name_title.split('-')[1])
        df_Company_Participants= pd.DataFrame( {'Participant name': Company_Participants_name, 'Company_Participants_type': Company_Participants_type})
        return df_Company_Participants
    find_Company_Participants=find_Company_Participants(df)
    def find_Conference_Call_Participants(df):
        i=0
        Company_Participants_list=[]
        Conference_Call_Participants_list=[]
        while i>=0:
            i=i+1
            if df_list[i]=='Conference Call Participants':
                break
            Company_Participants_list.append(i)   
        while i==i:
            i=i+1
            if len(df_list[i])>100:
                break
            Conference_Call_Participants_list.append(i)
        Company_Participants_list=Company_Participants_list[1:]
        Conference_Call_Participants_list=Conference_Call_Participants_list[:-1]
        Conference_Call_Participants_name=[]
        Conference_Call_Participants_type=[]
        for i in Conference_Call_Participants_list:
            name_title=df_list[i]
            Conference_Call_Participants_name.append((name_title.split('-')[0]))
            Conference_Call_Participants_type.append(name_title.split('-')[1])
        df_Conference_Call_Participants= pd.DataFrame( {'Participant name': Conference_Call_Participants_name, 'Participant_Organization': Conference_Call_Participants_type})
        return df_Conference_Call_Participants
    find_Conference_Call_Participants=find_Conference_Call_Participants(df)

    df_new=pd.DataFrame({'Speech': df_list,'Participant name':'','Section':'presentation'})
    #####################name
    name_index=[]
    for i in find_Company_Participants['Participant name']:
        name=(str(i)[:-1])
        a=[i for i,x in enumerate(df_list) if x==name]
        name_index=name_index+a
    for i in find_Conference_Call_Participants['Participant name']:
        name=(str(i)[:-1])
        a=[i for i,x in enumerate(df_list) if x==name]
        name_index=name_index+a
    Operator=[i for i,x in enumerate(df_list) if x=='Operator']
    name_index=name_index+Operator
    name_index.sort()
    for i in name_index:
        df_new.iloc[i:,1]=df_new.iloc[i,0]
   ####################QA
    QA_index=[i for i,x in enumerate(df_list) if x=='Question-and-Answer Session']
    for i in QA_index:
        df_new.iloc[i:,2]='Question-and-Answer Session'
    
        
        
        
    df_new=df_new[df_new['Speech'].str.len()>110]
    df_new['Ticker']=find_company['Ticker'][0]
    df_new['Company']=find_company['Company'][0]
    
    
    clean_2=list(filter(lambda a: a != '',df_list[0].split(')')[1].split(' ')))
    df_new['Title']=listToString(clean_2[:-7])
    
    
    clean_1 =list(filter(lambda a: a != '', df_list[0].split(')')[1].split(' ') ))
    df_new['Date']=listToString(clean_1[-6:-3])
    df_new['Time']=listToString(df_list[0].split(')')[1].split(' ')[-3:])    
    df_new['Participant name']=df_new['Participant name']+" "
    
    df_new=df_new[['Ticker','Company','Title','Date','Time','Section','Speech','Participant name']]
    #####add Participant_Organization/Company_Participants_type
    df_new=pd.merge(df_new,find_Company_Participants, how='left', left_on=['Participant name'], right_on=['Participant name'])
    df_new=pd.merge(df_new,find_Conference_Call_Participants, how='left', left_on=['Participant name'], right_on=['Participant name'])
    #prow=prow.append(df_new)
    
    return df_new

In [5]:
i=0
while i<len(df['Content']):
    i=i+1
    try:
        prow =prow.append(run(df['Content'][i-1]))
    except IndexError:
        pass

In [6]:
prow['Company_Participants_type']=prow['Company_Participants_type'].replace(np.nan, 'attendee')
display(prow)

prow.to_csv("final_dataset_demo.csv",index=False)

,Ticker,Company,Title,Date,Time,Speech,Participant name,Section,Company_Participants_type,Participant_Organization
0,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,"Hello, ladies and gentlemen. Thank you for sta...",Operator,presentation,attendee,NaN
1,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,"I will now turn the call over to your host, Mr...",Operator,presentation,attendee,NaN
2,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,"Thank you. Hello, everyone, and welcome to the...",Harry He,presentation,Investor Relations Director,NaN
3,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,"Dr. Yi Wang, our CEO and Founder and Mr. Bing ...",Harry He,presentation,Investor Relations Director,NaN
4,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,Please note that today's discussion will conta...,Harry He,presentation,Investor Relations Director,NaN
5,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,Some of these risks are beyond the company's c...,Harry He,presentation,Investor Relations Director,NaN
6,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,"During today's call, management will also disc...",Harry He,presentation,Investor Relations Director,NaN
7,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,"During the third quarter, total net revenue we...",Yi Wang,presentation,"Chief Executive Officer, Co",NaN
8,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,"During the quarter, we continued to maintain p...",Yi Wang,presentation,"Chief Executive Officer, Co",NaN
9,LAIX,LAIX Inc.,Q3 2020 Earnings Conference,"November 26, 2020",8:00 PM ET,"With that quick financial overview, I would no...",Yi Wang,presentation,"Chief Executive Officer, Co",NaN
